## Sequential Ensembles of Weak Learners

<img src="img/sequential.jpg">

WEAK LEARNERS

Si bien la definición precisa de la fortaleza de los alumnos (learners) se basa en la teoría del aprendizaje automático, para nuestros propósitos, un alumno fuerte es un buen modelo (o estimador). Por el contrario, un alumno débil (weak learner) es un modelo muy simple que no funciona tan bien. El único requisito de un alumno débil (para la clasificación binaria) es que se desempeñe mejor que las conjeturas aleatorias. 
Dicho de otra manera, su precisión debe ser solo un poco mejor que el 50%. Los árboles de decisión se utilizan a menudo como estimadores base para conjuntos secuenciales. Los algoritmos de impulso suelen utilizar tocones de decisión o árboles de decisión de profundidad 1

### AdaBoost: Adaptive boosting

AdaBoost es un algoritmo adaptativo: en cada iteración, entrena un nuevo estimador base que corrige los errores cometidos por el estimador base anterior. Por lo tanto, necesita alguna forma de garantizar que el algoritmo de aprendizaje base priorice los ejemplos de entrenamiento mal clasificados. AdaBoost hace esto manteniendo pesos sobre ejemplos de entrenamiento individuales. Intuitivamente, los pesos reflejan la importancia relativa de los ejemplos de entrenamiento. Los ejemplos clasificados incorrectamente tienen pesos más altos, mientras que los ejemplos clasificados correctamente tienen pesos más bajos. Cuando entrenamos el siguiente estimador base secuencialmente, los pesos permitirán que el algoritmo de aprendizaje priorice (y con suerte corrija) los errores de la iteración anterior. Este es el componente adaptativo de AdaBoost, que en última instancia conduce a un conjunto poderoso.

Training an ensemble of weak learners using AdaBoost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np


def fit_boosting(X, y, n_estimators=10):
    n_samples, n_features = X.shape
    D = np.ones((n_samples, )) # Nonnegative weights, initialized to 1
    estimators = []
    for t in range(n_estimators):
        D = D / np.sum(D) # Normalizes the weights so they sum to 1
        h = DecisionTreeClassifier(max_depth=1)
        h.fit(X, y, sample_weight=D) # Trains a weak learner (h_t) with weighted examples
        ypred = h.predict(X)
        # Computes the training error (Epsilon_t) andthe weight (Epsilon_t) of the weak learner
        e = 1 - accuracy_score(y, ypred, sample_weight=D) 
        a = 0.5 * np.log((1 - e) / e)
        # Updates the example weights: increase for misclassified examples, decrease for correctly classified examples
        m = (y == ypred) * 1 + (y != ypred) * -1
        D *= np.exp(-a * m)
        estimators.append((a, h)) # Saves the weak learner and its weight
    return estimators

Making predictions with AdaBoost

In [ ]:
def predict_boosting(X, estimators):
    pred = np.zeros((X.shape[0], )) # Initializes all the predictions to 0
    for a, h in estimators:
        pred += a * h.predict(X) # Makes weighted prediction for each example
    y = np.sign(pred) # Converts weighted predictions to –1/1 labels
    return y

In [ ]:
# Example

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split


# Generates a synthetic classification data set of 200 points
X, y = make_moons(n_samples=200, noise=0.1, random_state=13)

y = (2 * y) - 1 # Converts 0/1 labels to –1/1 labels

# Splits into training and test sets
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=13)

estimators = fit_boosting(Xtrn, ytrn) # Trains an AdaBoost model

ypred = predict_boosting(Xtst, estimators) # Makes predictions with this AdaBoost

# Podemos calcular la precisión general del conjunto de pruebas de nuestro modelo
from sklearn.metrics import accuracy_score

tst_err = 1 - accuracy_score(ytst, ypred)

print(tst_err)

AdaBoost with scikit-learn

In [ ]:
# Ejemplo
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


X, y = load_breast_cancer(return_X_y=True)
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=13)

from sklearn.ensemble import AdaBoostClassifier

shallow_tree = DecisionTreeClassifier(max_depth=2)
ensemble = AdaBoostClassifier(
    base_estimator=shallow_tree, # The base-learning algorithm AdaBoost uses to train weak learners
    n_estimators=20, # The number of weak learners that will be trained sequentially by AdaBoost.
    learning_rate=0.75 # An additional parameter that progressively shrinks the contribution of each successive weak learner trained for the ensemble.
    )
ensemble.fit(Xtrn, ytrn)

ypred = ensemble.predict(Xtst)
err = 1 - accuracy_score(ytst, ypred)
print(err)

AdaBoost with scikit-learn (MULTICLASS CLASSIFICATION)

scikit-learn contiene la implementación multiclase de AdaBoost llamada Stagewise Additive Modeling usando pérdida exponencial multiclase, o SAMME. 
SAMME es una generalización del algoritmo de refuerzo adaptativo de Freund y Schapire de dos a múltiples clases. 
Además de SAMME, AdaBoostClassifier también ofrece una variante denominada SAMME.R. 
La diferencia clave entre estos dos algoritmos es que SAMME.R maneja predicciones de valor real de algoritmos de estimación base (es decir, probabilidades de clase), mientras que Vanilla SAMME maneja predicciones discretas (es decir, etiquetas de clase).

In [ ]:
# Ejemplo
from sklearn.datasets import load_iris
from sklearn.utils.multiclass import unique_labels


X, y = load_iris(return_X_y=True)
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=13)
ensemble = AdaBoostClassifier(base_estimator=shallow_tree, 
                              n_estimators=20, 
                              learning_rate=0.75, algorithm='SAMME.R')
ensemble.fit(Xtrn, ytrn)
ypred = ensemble.predict(Xtst)
err = 1 - accuracy_score(ytst, ypred)
print(err)

* Cross validation to select the best *LEARNING RATE*

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np


n_learning_rate_steps, n_folds = 10, 10 
# Sets up stratified 10-fold CV and initializes the search space
learning_rates = np.linspace(0.1, 1.0, num=n_learning_rate_steps)
splitter = StratifiedKFold(n_splits=n_folds, shuffle=True)
trn_err = np.zeros((n_learning_rate_steps, n_folds))
val_err = np.zeros((n_learning_rate_steps, n_folds))
stump = DecisionTreeClassifier(max_depth=1) # Uses decision stumps as weak learners
for i, rate in enumerate(learning_rates): # For all choices of learning rates
    for j, (trn, val) in enumerate(splitter.split(X, y)): # For training, validation sets
        model = AdaBoostClassifier(algorithm='SAMME', base_estimator=stump, n_estimators=10, learning_rate=rate)
        model.fit(X[trn, :], y[trn]) # Fits a model to training data in this fold
        trn_err[i, j] = 1 - accuracy_score(y[trn], model.predict(X[trn, :])) # Computes training and validation /
        val_err[i, j] = 1 - accuracy_score(y[val], model.predict(X[val, :])) # errors for this fold
trn_err = np.mean(trn_err, axis=1) # Averages training and validation /
val_err = np.mean(val_err, axis=1) # errors across the folds

* Cross validation to select the *best number of weak learners*

In [ ]:
n_estimator_steps, n_folds = 5, 10
# Sets up stratified 10-fold CV and initializes the search space
number_of_stumps = np.arange(5, 50, n_estimator_steps)
splitter = StratifiedKFold(n_splits=n_folds, shuffle=True)
trn_err = np.zeros((len(number_of_stumps), n_folds))
val_err = np.zeros((len(number_of_stumps), n_folds))
stump = DecisionTreeClassifier(max_depth=1) # Uses decision stumps as weak learners
for i, n_stumps in enumerate(number_of_stumps): # For all estimator sizes
    for j, (trn, val) in enumerate(splitter.split(X, y)): # For training, validation sets
        model = AdaBoostClassifier(algorithm='SAMME', 
                                   base_estimator=stump, 
                                   n_estimators=n_stumps, 
                                   learning_rate=1.0)
        model.fit(X[trn, :], y[trn]) # Fits a model to training data in this fold
        trn_err[i, j] = 1 - accuracy_score(y[trn], model.predict(X[trn, :])) # Computes the training and validation /
        val_err[i, j] = 1 - accuracy_score(y[val], model.predict(X[val, :])) # errors for this fold
trn_err = np.mean(trn_err, axis=1)
val_err = np.mean(val_err, axis=1) # Averages the errors across the folds

## LogitBoost: Boosting with the logistic loss

Diferencias respecto a AdaBoost:
* *Función de pérdida:* AdaBoost utiliza la función de pérdida exponencial, que se enfoca en corregir los ejemplos mal clasificados en cada iteración. Por otro lado, LogitBoost utiliza una función de pérdida logística, que se basa en el modelo logit y tiene como objetivo ajustar los pesos de los ejemplos de acuerdo con las probabilidades estimadas.
* *Base estimator:* AdaBoost puede utilizar cualquier algoritmo de aprendizaje automático como base estimator, mientras que LogitBoost utiliza modelos de regresión logística como base estimator en cada iteración. Esto significa que LogitBoost está diseñado específicamente para problemas de clasificación binaria.
* *Predicciones:* AdaBoost utiliza la suma ponderada de las predicciones de todos los clasificadores débiles para generar la predicción final. En contraste, LogitBoost utiliza las probabilidades estimadas por los clasificadores débiles y las combina mediante una regresión logística para obtener la predicción final.
* *Interpretación probabilística:* LogitBoost proporciona una interpretación probabilística directa de las predicciones, ya que utiliza una función de pérdida logística y estima probabilidades. En cambio, AdaBoost no proporciona directamente una interpretación probabilística, ya que se basa en la función de pérdida exponencial.


LogitBoost for classification

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from scipy.special import expit


def fit_logitboosting(X, y, n_estimators=10):
    n_samples, n_features = X.shape
    D = np.ones((n_samples, )) / n_samples
    p = np.full((n_samples, ), 0.5) # Initializes example weights, “pred” probabilities
    estimators = []
    for t in range(n_estimators):
        z = (y - p) / (p * (1 - p)) # Computes working responses
        D = p * (1 - p) # Computes new example weights
        h = DecisionTreeRegressor(max_depth=1) # Use decision-tree regression as base estimators for classification problems
        h.fit(X, z, sample_weight=D)
        estimators.append(h) # Appends weak learners to ensemble Ft+1(x) = Ft(x) + ht(x)
        if t == 0:
            margin = np.array([h.predict(X) for h in estimators]).reshape(-1, )
        else:
            margin = np.sum(np.array([h.predict(X) for h in estimators]), axis=0)
        p = expit(margin) # Updates prediction probabilities
    return estimators

LogitBoost for prediction

In [ ]:
def predict_logit_boosting(X, estimators):
    pred = np.zeros((X.shape[0], ))
    for h in estimators:
        pred += h.predict(X)
    y = (np.sign(pred) + 1) / 2 # Converts –1/1 predictions to 0/1
    return y